<a href="https://colab.research.google.com/github/vedant04L/SMS_Prediction/blob/main/Okra_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from tensorflow.keras.preprocessing import image

# Set paths to your dataset
base_dir = '/content/drive/My Drive/okra sample images'
type_categories = ['leaf', 'fruit', 'flower']

# Secondary classification categories
leaf_categories = ['fresh okra leaf', 'powdery mildew', 'leaf spot', 'leaf curl']
fruit_categories = ['healthy fruit', 'fruit rot', 'fruit borer']
flower_categories = ['healthy flower', 'aphid on flower']

# Image data generator for data augmentation
type_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Prepare the type classification datasets
type_train_generator = type_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    classes=type_categories,
    class_mode='categorical',
    subset='training'
)

type_validation_generator = type_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    classes=type_categories,
    class_mode='categorical',
    subset='validation'
)

# Model for classifying type (leaf, fruit, flower)
type_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(type_categories), activation='softmax')
])

type_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the type model
type_model.fit(
    type_train_generator,
    epochs=10,
    validation_data=type_validation_generator,
    verbose=1
)

# Save the type model
type_model.save('okra_type_classification_model.h5')

# Function to create model for disease classification
def create_secondary_model(num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Train and save models for each type-specific classification
def train_and_save_secondary_model(base_dir, category, categories, model_name):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(150, 150),
        batch_size=32,
        classes=categories,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(150, 150),
        batch_size=32,
        classes=categories,
        class_mode='categorical',
        subset='validation'
    )

    # Ensure generators have data
    if train_generator.samples == 0 or validation_generator.samples == 0:
        raise ValueError(f"No data found in {base_dir}. Please check the directory structure and ensure it contains images.")

    model = create_secondary_model(len(categories))
    model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        verbose=1
    )
    model.save(model_name)

# Train models for leaf, fruit, and flower
train_and_save_secondary_model(f'{base_dir}/leaf', 'leaf', leaf_categories, 'okra_leaf_disease_classification_model.h5')
train_and_save_secondary_model(f'{base_dir}/fruit', 'fruit', fruit_categories, 'okra_fruit_disease_classification_model.h5')
train_and_save_secondary_model(f'{base_dir}/flower', 'flower', flower_categories, 'okra_flower_disease_classification_model.h5')

# Function to load and predict an image type
def predict_type(img_path):
    # Load the type model
    model = tf.keras.models.load_model('okra_type_classification_model.h5')

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Predict
    prediction = model.predict(img_array)
    return type_categories[np.argmax(prediction)]

# Function to load appropriate model for disease classification
def predict_disease(img_path, type_pred):
    if type_pred == 'leaf':
        disease_categories = leaf_categories
        model_path = 'okra_leaf_disease_classification_model.h5'
    elif type_pred == 'fruit':
        disease_categories = fruit_categories
        model_path = 'okra_fruit_disease_classification_model.h5'
    elif type_pred == 'flower':
        disease_categories = flower_categories
        model_path = 'okra_flower_disease_classification_model.h5'
    else:
        return "Unknown type"

    # Load the specific model for the detected type
    disease_model = create_secondary_model(len(disease_categories))
    disease_model.load_weights(model_path)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Predict the disease
    prediction = disease_model.predict(img_array)
    return disease_categories[np.argmax(prediction)]

# Example usage
img_path = '/content/3.jpg'
type_result = predict_type(img_path)
print(f'The predicted type is: {type_result}')
disease_result = predict_disease(img_path, type_result)
print(f'The predicted disease is: {disease_result}')


Found 69 images belonging to 3 classes.
Found 16 images belonging to 3 classes.
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5058 - loss: 2.1869 - val_accuracy: 0.2500 - val_loss: 1.6484
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.4200 - loss: 1.6802 - val_accuracy: 0.2500 - val_loss: 1.0895
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.3687 - loss: 1.0386 - val_accuracy: 0.6875 - val_loss: 0.8249
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6503 - loss: 0.8774 - val_accuracy: 0.6875 - val_loss: 0.7841
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6601 - loss: 0.7701 - val_accuracy: 0.6875 - val_loss: 0.7694
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6425 - loss: 0.7100 - val_accuracy: 0.6875 - val_loss: 0.9675
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.6371 - loss: 0.8086 - val_accuracy: 0.6875 - val_loss: 0.7969
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy

Found 34 images belonging to 4 classes.
Found 8 images belonging to 4 classes.
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.1893 - loss: 1.3434 - val_accuracy: 0.6250 - val_loss: 5.5233
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.5784 - loss: 5.7888 - val_accuracy: 0.6250 - val_loss: 0.9248
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5104 - loss: 1.3773 - val_accuracy: 0.3750 - val_loss: 1.1526
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.5000 - loss: 1.1774 - val_accuracy: 0.6250 - val_loss: 0.9455
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 856ms/step - accuracy: 0.6501 - loss: 0.8802 - val_accuracy: 0.6250 - val_loss: 0.9208
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.5784 - loss: 1.0193 - val_accuracy: 0.6250 - val_loss: 0.9159
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.7451 - loss: 0.7191 - val_accuracy: 0.6250 - val_loss: 0.8473
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 860ms/step - accur

Found 17 images belonging to 3 classes.
Found 3 images belonging to 3 classes.
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3529 - loss: 1.0657 - val_accuracy: 0.3333 - val_loss: 3.5575
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4706 - loss: 3.5003 - val_accuracy: 0.3333 - val_loss: 3.3990
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2941 - loss: 3.3475 - val_accuracy: 0.6667 - val_loss: 1.0557
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3529 - loss: 1.2234 - val_accuracy: 0.3333 - val_loss: 0.9960
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5882 - loss: 0.9484 - val_accuracy: 0.3333 - val_loss: 0.9377
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5294 - loss: 0.8698 - val_accuracy: 0.6667 - val_loss: 0.8170
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7059 - loss: 0.7685 - val_accuracy: 0.6667 - val_loss: 0.7176
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0

Found 8 images belonging to 2 classes.
Found 1 images belonging to 2 classes.
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.2500 - loss: 0.7240 - val_accuracy: 1.0000 - val_loss: 0.0030
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 2.2384 - val_accuracy: 0.0000e+00 - val_loss: 2.5047
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 1.6510 - val_accuracy: 1.0000 - val_loss: 0.6818
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 650ms/step - accuracy: 0.6250 - loss: 0.4859 - val_accuracy: 1.0000 - val_loss: 0.2600
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8750 - loss: 0.3070 - val_accuracy: 1.0000 - val_loss: 0.2213
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.2072 - val_accuracy: 1.0000 - val_loss: 0.2045
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step - accuracy: 1.0000 - loss: 0.1351 - val_accuracy: 1.0000 - val_loss: 0.1039
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 641ms/step -

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
The predicted type is: fruit
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
The predicted disease is: fruit rot
